In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from scipy import stats

In [2]:
def breaker() -> None: print("\n" + 50*"*" + "\n")
    
MODELS_PATH:str = "../input/4ac-swinfs-ml-train/models"

In [3]:
swin_B_model_paths = [os.path.join(MODELS_PATH, name) for name in os.listdir(MODELS_PATH) if re.match(r"swin_base_patch4_window12_384_in22k", name, re.IGNORECASE)]
swin_L_model_paths = [os.path.join(MODELS_PATH, name) for name in os.listdir(MODELS_PATH) if re.match(r"swin_large_patch4_window12_384_in22k", name, re.IGNORECASE)]

swin_B_features = np.load("../input/4actrsfsna384/swin_base_patch4_window12_384_in22k.npy")[-729:]
swin_L_features = np.load("../input/4actrsfsna384/swin_large_patch4_window12_384_in22k.npy")[-729:]

In [4]:
y_pred_1 = np.zeros((1, swin_B_features.shape[0]), dtype=np.uint8)

for swin_B_model_path in swin_B_model_paths:
    with open(swin_B_model_path, "rb") as fp: model = pickle.load(fp)
    temp = model.predict(swin_B_features)
    y_pred_1 = np.concatenate((y_pred_1, temp.reshape(1, -1)), axis=0)

y_pred_1 = y_pred_1[1:]

In [5]:
y_pred_2 = np.zeros((1, swin_L_features.shape[0]), dtype=np.uint8)

for swin_L_model_path in swin_L_model_paths:
    with open(swin_L_model_path, "rb") as fp: model = pickle.load(fp)
    temp = model.predict(swin_L_features)
    y_pred_2 = np.concatenate((y_pred_2, temp.reshape(1, -1)), axis=0)

y_pred_2 = y_pred_2[1:]

In [6]:
y_pred = np.concatenate((y_pred_1, y_pred_2), axis=0)
y_pred = stats.mode(y_pred)[0]

df = pd.read_csv("../input/4-animal-classification/Sample_submission.csv")
df["Label"] = y_pred.reshape(-1)
df.to_csv("submission.csv", index=False)